# Adadelta

Adadelta是AdaGrad的又一种变体，前者减少了学习率适应坐标的数量。

广义上Adadelta没有学习率，它使用变化量本身作为未来变化的校准，它是在2012年被提出的

使用连个状态变量$s_t$存储梯度二阶导数的泄露平均值，$\Delta \mathbf{x}$来存储模型本身中参数变化二阶导数的泄露平均值。
下面的命名都是作者文章中的原始符号，其实没有必要替换掉与AdaGrad等方法中的用途一致的参数。

## 细节
使用RMSProp类似的泄露更新：
$$
s_t = \rho s_{t-1} + (1-\rho)g_t^2
$$

然后调整梯度：

$$
g_t^\prime = \frac {\sqrt{\Delta x_{t-1} + \epsilon}}{\sqrt{s_t + \epsilon}} \odot g_t
$$

其中$ \Delta x_{t-1}$是重新缩放梯度$g_t^\prime$的泄露平均值，将$ \Delta x_0$初始化为0，然后在每个步骤中使用$g_t^\prime$去更新它，即：
$$
\Delta x_t = \rho \Delta x_{t-1} + (1-\rho){g_t^\prime}^2
$$

同样地，在分母分子上面加上epsilon是为了增加数值稳定性

In [ ]:
# 代码实现
import torch as t

def init_adadelta_states(features_dim):
    s_w,s_b = t.zeros((features_dim,1)),t.zeros(1)
    delta_w,delta_b = t.zeros((features_dim,1)),t.zeros(1)
    return ((s_w,s_b),(delta_w,delta_b))

def adadelta(params,states,hyperparams):
    rho,eps = hyperparams["rho"],1e-5
    for p,(s,delta) in zip (params,states):
        with t.no_grad():
            s[:] = rho * s + (1-rho)*t.square(p.grad)
            g = (t.sqrt(delta+eps)/t.sqrt(s+eps))*p.grad
            p[:] -=g
            delta[:]=rho*delta +(1-rho)*g*g
        p.grad.data.zero_()